# EL-HAMDAOUI MAROUANE | IAGI | CI-2
# TP 4 : Espace d'États & Recherche BFS / DFS - Algorithmes de Parcours

# Objectifs du TP
- Comprendre la notion d'espace d'états pour résoudre des problèmes 
- Modéliser un problème classique (Fermier-Loup-Chèvre-Laitue) 
- Implémenter l'algorithme BFS (Breadth-First Search / Parcours en largeur) 
- Implémenter l'algorithme DFS (Depth-First Search / Parcours en profondeur) 
- Comparer les performances et caractéristiques des deux approches

# Problème Étudié
## Le Fermier, le Loup, la Chèvre et la Laitue

**Énoncé :**
Un fermier doit traverser une rivière avec : 
- Un loup (L) 
- Une chèvre (C) 
- Une laitue (T pour "Turnip") 
Règles : 
- Le bateau peut transporter le fermier + un seul objet à la fois 
- La présence du fermier empêche tout danger 
Contraintes : 
- Le loup ne peut pas rester seul avec la chèvre (sinon il la mange) 
- La chèvre ne peut pas rester seule avec la laitue (sinon elle la mange) 
Objectif : Faire traverser tout le monde de l'ouest vers l'est sans violer les contraintes. 

## Partie 1 : Modélisation du Problème 

Dans cette partie, vous allez modéliser le problème en définissant les états, les transitions et 
les contraintes. 
 
### Question 1 : États de référence et affichage 

Un état est représenté par un tuple de 4 valeurs (F, L, C, T) où chaque valeur vaut 0 (ouest) 
ou 1 (est). 

- A) Définissez les constantes et états initiaux :
````python 
OUEST, EST =  
INITIAL =               # Tous à l'ouest 
GOAL =                  # Tous à l'est ²
NOMS =
```

In [25]:
OUEST, EST = 0, 1
INITIAL = (0, 0, 0, 0)
GOAL = (1, 1, 1, 1)
NOMS = ('F', 'L', 'C', 'T')

- B) Implémentez la fonction `affiche_etat(s)`:
 
Cette fonction doit afficher un état sous la forme: 

`Ouest: [F, L] | Est: [C, T]`  

In [26]:
def affiche_etat(s):
    ouest_state = [NOMS[i] for i in range(len(s)) if s[i] == 0]
    est_state = [NOMS[i] for i in range(len(s)) if s[i] == 1]
    print(f"Ouest: {ouest_state} | Est: {est_state}")

affiche_etat(INITIAL)
affiche_etat(GOAL)

Ouest: ['F', 'L', 'C', 'T'] | Est: []
Ouest: [] | Est: ['F', 'L', 'C', 'T']


### Question 2 : Test d'état interdit

Implémentez la fonction `est_interdit(s)` qui retourne True si l'état viole une contrainte 

**Exemples de tests :**
```python 
(0, 0, 0, 0) -> False 
(1, 0, 0, 0) -> True 
(1, 1, 0, 0) -> True 
(1, 0, 1, 0) -> False 
(0, 1, 0, 1) -> False
```

In [27]:
def est_interdit(s):
    return s == None or (s[0] != s[1] == s[2]) or (s[0] != s[2] == s[3])

#exemples de tests
print(est_interdit((0, 0, 0, 0))) # -> False
print(est_interdit((1, 0, 0, 0))) # -> True
print(est_interdit((1, 1, 0, 0))) # -> True
print(est_interdit((1, 0, 1, 0))) # -> False
print(est_interdit((0, 1, 0, 1))) # -> False
print(est_interdit((1, 0, 0, 1))) # -> True

False
True
True
False
False
True


### Question 3 : Opérateurs (actions) 

Implémentez la fonction `appliquer_action(s, avec=None)` qui simule une traversée :

- Le fermier traverse toujours (change de côté) 
- Si avec='L', 'C' ou 'T', l'objet correspondant traverse aussi (s'il est du même côté que le 
fermier) 
- Si avec=None, le fermier traverse seul 
- Retourner None si l'action est impossible (objet pas du même côté)

**Exemples :**

```python 
appliquer_action((0,0,0,0), None) → (1,0,0,0) 
appliquer_action((0,0,0,0), 'C') → (1,0,1,0) 
appliquer_action((0,1,0,0), 'L') → None (L pas du même côté que F)
```

In [31]:
def appliquer_action(s, avec=None):
    result = [etat for etat in s]
    result[0] = (result[0] + 1) % 2
    
    if avec == None:        
        return tuple(result)

    avec_index = NOMS.index(avec)
    
    if s[0] != s[avec_index] or len(avec) != 1:
        return None
    
    result[avec_index] = (result[avec_index] + 1) % 2

    return tuple(result)

# examples:
print(appliquer_action((0,0,0,0), None)) # → (1,0,0,0)
print(appliquer_action((0,0,0,0), 'C'))  # → (1,0,1,0)
print(appliquer_action((0,1,0,0), 'L'))  # → None

(1, 0, 0, 0)
(1, 0, 1, 0)
None


### Question 4 : Successeurs valides

Implémentez la fonction `successeurs(s)` qui retourne la liste de tous les états successeurs valides depuis l'état s :

1. Essayer toutes les actions possibles : None, 'L', 'C', 'T' 
2. Appliquer chaque action avec appliquer_action 
3. Ne garder que les états non-None, non-interdits, et non-dupliqués 
 
Testez avec l'état initial et affichez les successeurs :

In [30]:
def successeurs(s):
    actions = [None, 'L', 'C', 'T']
    result = []
    
    for action in actions:
        curr_state = appliquer_action(s, action)

        if not est_interdit(curr_state):
            result.append(curr_state)

    return result

# examples
print(successeurs(INITIAL))
print(successeurs((1,0,1,0)))

[(1, 0, 1, 0)]
[(0, 0, 1, 0), (0, 0, 0, 0)]


## Partie 2 : Algorithmes BFS & DFS 

Vous allez maintenant implémenter deux algorithmes classiques de parcours de graphe pour résoudre le problème.

### Question 5 : Implémentation de BFS (Breadth-First Search)

**Principe :** BFS explore les états niveau par niveau, garantissant de trouver le chemin le plus court (en nombre d'étapes).

Implémentez la fonction `bfs(start)` qui retourne `(chemin, nb_explorations)`:

In [ ]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
    def add_child(self, child_node):
        self.children.append(child_node)

def bfs(start):
    visited = set()
    chemin = []
    start_node = Node(start)
    
    queue = []
    queue.append(start_node)
    visited.add(start_node.state)
    nb_explorations = 0

    while queue:
        current_node = queue.pop(0)
        nb_explorations += 1

        if current_node.state == GOAL:
            while current_node:
                chemin.insert(0, current_node.state)
                current_node = current_node.parent
            
            return (chemin, nb_explorations)

        for succ in successeurs(current_node.state):
            if succ not in visited:
                visited.add(succ)
                child_node = Node(succ, current_node)
                current_node.add_child(child_node)
                queue.append(child_node)

    return (None, nb_explorations)

Chemin trouvé par BFS en 7 étapes et 10 explorations :
Ouest: ['F', 'L', 'C', 'T'] | Est: []
Ouest: ['L', 'T'] | Est: ['F', 'C']
Ouest: ['F', 'L', 'T'] | Est: ['C']
Ouest: ['T'] | Est: ['F', 'L', 'C']
Ouest: ['F', 'C', 'T'] | Est: ['L']
Ouest: ['C'] | Est: ['F', 'L', 'T']
Ouest: ['F', 'C'] | Est: ['L', 'T']
Ouest: [] | Est: ['F', 'L', 'C', 'T']


# Question 6 : Implémentation de DFS (Depth-First Search) 

**Principe :** DFS explore en profondeur, ne garantit pas le chemin le plus court mais peut être plus rapide en mémoire. 

Implémentez la fonction `dfs(start, limit=None)` qui retourne `(chemin, nb_explorations)` :

In [ ]:
def dfs(start, limit=None):
    visited = set()
    chemin = []
    start_node = Node(start)
    
    stack = []
    stack.append((start_node, 0))
    visited.add(start_node.state)
    nb_explorations = 0

    while stack:
        current_node, depth = stack.pop()
        nb_explorations += 1

        if current_node.state == GOAL:
            while current_node:
                chemin.insert(0, current_node.state)
                current_node = current_node.parent
            
            return (chemin, nb_explorations)

        if limit is None or depth < limit:
            for succ in successeurs(current_node.state):
                if succ not in visited:
                    visited.add(succ)
                    child_node = Node(succ, current_node)
                    current_node.add_child(child_node)
                    stack.append((child_node, depth + 1))
                    
    return (None, nb_explorations)

### Question 7 : Exécution et comparaison 

Exécutez les deux algorithmes sur le problème et remplissez le tableau suivant :

```
Critère                |  BFS          |  DFS
Longueur du chemin     |               |  
Nombre de traversées   |               |
```

In [38]:
# Approache BFS
bfs_chemin, bfs_explorations = bfs(INITIAL)
print("BFS Chemin:", bfs_chemin)
print("BFS Nombre d'explorations:", bfs_explorations)

# Approche DFS
dfs_chemin, dfs_explorations = dfs(INITIAL)
print("DFS Chemin:", dfs_chemin)
print("DFS Nombre d'explorations:", dfs_explorations)

BFS Chemin: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 1, 1, 0), (0, 1, 0, 0), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]
BFS Nombre d'explorations: 10
DFS Chemin: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 0, 1, 1), (0, 0, 0, 1), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]
DFS Nombre d'explorations: 9


tableau:
```
Critère               |  BFS          |  DFS
Longueur du chemin    | 8             | 8  
Nombre de traversées  | 10            | 9